In [2]:
import os
os.chdir('/home/guohao826/AppAlgorithmFace')
import cv2
import uuid
import base64
import time
import logging
import tempfile
import numpy as np
from src.config import Config
from src.service.alive_alg.alive import LiveFaceDetection, check_alive
from src.service.face_alg.detect_inference import DetectInference
from src.service.face_alg.landmark_inference import LandmarkInference
from src.service.face_alg.antispoofing_inference import AntiSpoofingInference

from src.service.face_alg.service_demo import PthExtractor, detect_align, get_sim
from src.service.image import ImageService
from src.util.decorator import catch
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

## 加载模型

In [3]:
detector = DetectInference(Config.DETECT_MODEL_PATH, Config.DETECT_GPUID, Config.DETECT_INPUT_SHAPE,
                               Config.DETECT_THRESHOLD)
landermark_detector = LandmarkInference(Config.DETECT_LANDMARK_MODEL_PATH, Config.DETECT_GPUID)
# 活体模型
# alive_model = LiveFaceDetection(Config.ALIVE_MODEL_PATH, Config.ALIVE_THRESHOLD_ORIENTATION,
#                                     Config.ALIVE_THRESHOLD_MOUSE, Config.ALIVE_THRESHOLD_MOVE,
#                                     Config.ALIVE_THRESHOLD_EYE, Config.ALIVE_THRESHOLD_ALIGH,
#                                     Config.ALIVE_WIN_WIDTH, Config.ALIVE_WIN_HEIGHT)
# 特征检测模型
feature_model = PthExtractor(Config.FEATURE_MODEL_PATH,
                             Config.FEATURE_MODEL_CONFIG,
                             Config.FEATURE_GPUID,
                             batch_size=Config.FEATURE_BATCH_SIZE,
                             im_shape=Config.FEATURE_INPUT_SHAPE)

In [4]:
##预处理
def preprocess(frame):
    # img_w = frame.shape[1]
    # img_h = frame.shape[0]
    # if img_w > img_h:
    #     frame = np.rot90(frame)

    dim = frame.shape
    # resize_w = alive_model.WIN_WIDTH
    # resize_h = int(resize_w * dim[0] / dim[1])

    # frame = cv2.resize(frame, (resize_w, resize_h))
    return frame

In [5]:
##检测人脸区域
def detect_face(frame):
    bboxes,lads = detector.detect([frame])
    bbox= bboxes[0]
    return bbox

# def plot_one_box(x, im, label=None, line_thickness=3):
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    color = [0,0,255]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    c3 = c1[0],c2[1]
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    
# def plot_face_region(bboxs,im):
    imm = im.copy()
    for bbox in bboxs:
        plot_one_box(bbox,imm)
    return imm
    
    
# def plot_lads(landermarks,frame):
    res1 = frame.copy()
    # for a in landermark_98.getA()  :
        # cv2.circle(res1,(int(a[0]),int(a[1])),2,(0,0,255),-1)
    return res1

# def get_mouse_status(landmarks_in):
    mouseArea = []
    for i in range(48, 60):
        point = landmarks_in[i]
        mouseArea.append([int(point[0, 0]), int(point[0, 1])])
    mouseArea = np.array(mouseArea)
    mouseSize = cv2.contourArea(mouseArea)
    mouseLength = landmarks_in[54][0, 0] - landmarks_in[48][0, 0]
    mouseRatio = mouseSize / (mouseLength * mouseLength)
    return mouseRatio

# def get_eyes_status(landmarks_in):

    landmarks_eye = landmarks_in[60:68]
    landmarks_right = landmarks_in[68:76]

    # left_err = alive_model.eye_aspect_ratio_eight_points(landmarks_eye)
    # right_err = alive_model.eye_aspect_ratio_eight_points(landmarks_right)
    mean_err = (landmarks_eye + landmarks_right) / 2.0
    return mean_err


# def getfaceVar(image,face_box, img_w, img_h):
    left_x = int(face_box[0])
    left_y = int(face_box[1])
    right_x = int(face_box[2])
    right_y = int(face_box[3])

    if left_x < 1:
        left_x = 0
    if left_y < 1:
        left_y = 0
    if right_x > (img_w - 1):
        right_x = (img_w - 1)
    if right_y > (img_h - 1):
        right_y = (img_h - 1)

    crop_face = image[left_y:right_y, left_x:right_x]
    # image = cv2.cvtColor(np.asarray(PIl_image),cv2.COLOR_RGB2BGR)
    img2gray = cv2.cvtColor(crop_face, cv2.COLOR_BGR2GRAY)
    imageVar = cv2.Laplacian(img2gray, cv2.CV_64F).var()
    return imageVar


def getImageVar(image):
    # image = cv2.cvtColor(np.asarray(PIl_image),cv2.COLOR_RGB2BGR)
    img2gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imageVar = cv2.Laplacian(img2gray, cv2.CV_64F).var()
    return imageVar


# def isLightnessover( frame, face_box, img_w, img_h):
    left_x = int(face_box[0])
    left_y = int(face_box[1])
    right_x = int(face_box[2])
    right_y = int(face_box[3])

    if left_x < 1:
        left_x = 0
    if left_y < 1:
        left_y = 0
    if right_x > (img_w - 1):
        right_x = (img_w - 1)
    if right_y > (img_h - 1):
        right_y = (img_h - 1)

    crop_face = frame[left_y:right_y, left_x:right_x]
    img_hsv = cv2.cvtColor(crop_face, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv)
    bright_value = np.mean(np.mean(v))

    # value from [0-255] 0 is dark
    if bright_value > 230:
        return False
    return True



In [6]:
dirpath = '/home/guohao826/AppAlgorithmFace/process_file/imgs'
root = '/home/guohao826/AppAlgorithmFace/process_file'
filenames = os.listdir(dirpath)
os.makedirs(os.path.join(root,'no_face'),exist_ok=True)
# os.makedirs(os.path.join(root,'mohu'),exist_ok=True)
# os.makedirs(os.path.join(root,'light_lack'),exist_ok=True)
# os.makedirs(os.path.join(root,'light_over'),exist_ok=True)
# os.makedirs(os.path.join(root,'small_face'),exist_ok=True)
# os.makedirs(os.path.join(root,'ola'),exist_ok=True)
# os.makedirs(os.path.join(root,'eye_close'),exist_ok=True)
# os.makedirs(os.path.join(root,'mouse_open'),exist_ok=True)

#### 关键点提取

In [ ]:
for filename in tqdm(filenames):
    try:
        #filename = 'base_20211219174036f5fd906f42ca40db82b532e07bf5215a.jpg'
        img_path = os.path.join(dirpath,filename)
        img = cv2.imread(img_path)
        #img = cv2.imread('./eye.jpg')
        frame = preprocess(img)
        bounding_boxes = detect_face(frame)
        dim = frame.shape
        img_h = dim[0]
        img_w = dim[1]
        if len(bounding_boxes) > 0:
            # index = alive_model.maxbox(bounding_boxes)
            # box = bounding_boxes[index]
            # isLightover = isLightnessover(frame, box, img_w, img_h)
            # isLightFace = alive_model.isLightnessEnough(frame, box, img_w, img_h)
            # isBigFace = alive_model.isBig(box, img_w, img_h)
            ImageVar = getImageVar(frame)
            if ImageVar < 10:
                target_path = os.path.join(root,'mohu',filename)
                shutil.copyfile(img_path,target_path)
            # if isLightFace == False:
            #     target_path = os.path.join(root,'light_lack',filename)
            #     shutil.copyfile(img_path,target_path)
            # if isBigFace == False:
            #     target_path = os.path.join(root,'small_face',filename)
            #     shutil.copyfile(img_path,target_path)
            # if isLightover == False:
            #     target_path = os.path.join(root,'light_over',filename)
            #     shutil.copyfile(img_path,target_path)
            ##关键点检测
            faceimg, face_size, (x1, y1) = landermark_detector.preprocess(max(bounding_boxes), frame)
            landmark_QC = landermark_detector.inference(faceimg, face_size)
            # landermark_98 = alive_model.landermark_plus_leftcorner(landmark_QC, np.array([x1, y1]))
            # landmarks = alive_model.index_98to68(landmark_QC, np.array([x1, y1]))
            # reprojectdst, euler_angle = alive_model.get_head_pose(landmarks)
            # aligned_sum = abs(euler_angle[0, 0]) + abs(euler_angle[1, 0]) + abs(euler_angle[2, 0])
            # eye_status = get_eyes_status(landermark_98)
            # mouse_status = get_mouse_status(landmarks)
            # if aligned_sum > 80 :
            #     target_path = os.path.join(root,'ola',filename)
            #     shutil.copyfile(img_path,target_path)
            # elif eye_status<0.11:
            #     target_path = os.path.join(root,'eye_close',filename)
            #     shutil.copyfile(img_path,target_path)
            # elif mouse_status>0.6:
            #     target_path = os.path.join(root,'mouse_open',filename)
            #     shutil.copyfile(img_path,target_path)

            for point in landmark_QC:
                cv2.circle(img, (int(point[0] + x1), int(point[1] + y1)), 2, (255, 0, 0), -1)
            output_folder = "/home/guohao826/AppAlgorithmFace/process_file/results1"
            result_img_path = os.path.join(output_folder, 'result_' + filename)
            cv2.imwrite(result_img_path, img)
        else:
            target_path = os.path.join(root,'no_face',filename)
            shutil.copyfile(img_path,target_path)
    except:
        print(filename)
            


## 人脸比对

In [11]:
def get_compare_score(feature_1, feature_2):
    '''
    根据特征计算相似度得分
    '''
    return get_sim(feature_1, feature_2)


def get_feature_value(base64_str=None, url=None, image=None):
    '''
    base64_str: 图片 base64 字符串
    url: 图片 url
    image: cv2 的 image 对象
    依次判断三个参数，任一个满足时开始进行特征提取
    返回识别的特征值
    '''
    # 处理二进制
    if base64_str is not None:
        binary = base64.b64decode(base64_str)
    elif url is not None:
        binary = ImageService.download_image(url)
    else:
        binary = None
    # 二进制或 image 对象需要至少存在一个
    if binary is None and image is None:
        return None
    filename = uuid.uuid4().hex
    if binary is not None:
        # buf = np.asarray(bytearray(binary), dtype="uint8")
        buf = np.frombuffer(binary, dtype="uint8")
        image = cv2.imdecode(buf, cv2.IMREAD_COLOR)
    logging.debug("detect start: %s", time.time())
    key_imarray_map = detect_align(detector, {filename: image}, align_shape=Config.DETECT_ALIGN_SHAPE)
    logging.debug("detect done: %s", time.time())
    feature_map = feature_model.inference(key_imarray_map)
    logging.debug("feature done: %s", time.time())
    return feature_map[filename]

### 导入视频进行人脸比对

In [12]:
#提取视频人脸帧
def process_video(frame):
    # 人脸检测
    frame = preprocess(frame)
    bounding_boxes = detect_face(frame)
    # 检测到人脸的标志
    face_detected = False
    if len(bounding_boxes) > 0:
        face_detected = True
    return face_detected


#对每一帧进行人脸比对
# async def process_frame(frame,base_img):
    detect_feature = await get_feature_value(image=frame)
    base_feature = await get_feature_value(image=base_img)
    data_score = get_compare_score(detect_feature, base_feature)
    # print(data_score)
    data_ismatch = True if data_score >= Config.FEATURE_MATCH_THRESHOLD else False
    return data_ismatch
        

SyntaxError: 'await' outside async function (318083820.py, line 15)

In [9]:
video_path = "/home/guohao826/AppAlgorithmFace/material/input_video/2.mp4"
output_folder = "/home/guohao826/AppAlgorithmFace/material/output_frame"
base_img_path = '/home/guohao826/AppAlgorithmFace/material/images/oppen.jpg'

#### 处理视频

In [ ]:
# 打开视频文件
video = cv2.VideoCapture(video_path)
# 获取视频的帧率
fps = int(video.get(cv2.CAP_PROP_FPS))
# 获取视频的宽度和高度
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
# 记录有人脸的帧数
frames_with_face = 0
# 创建输出文件夹
os.makedirs(output_folder, exist_ok=True)
# 遍历视频的每一帧
while True:
    ret, frame = video.read()
    # 如果没有帧了，退出循环
    if not ret:
        break
    # 人脸检测
    face_detected = process_video(frame)
    # 保存有人脸的帧
    if face_detected:
        frames_with_face += 1
        frame_filename = f"frame_{frames_with_face}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)
        cv2.imwrite(frame_path, frame)
# 释放视频流
video.release()
print(f"Total frames with face: {frames_with_face}")

#### 人脸比对

In [13]:
#人脸比对
base_img = cv2.imread(base_img_path)
framenames = os.listdir(output_folder)
for framename in framenames:
    try:
        frame_path = os.path.join(output_folder,framename)
        frame = cv2.imread(frame_path)
        detect_feature = get_feature_value(image=frame)
        base_feature = get_feature_value(image=base_img)
        data_score = get_compare_score(detect_feature, base_feature)
        # print(data_score)
        data_ismatch = True if data_score >= Config.FEATURE_MATCH_THRESHOLD else False
        if data_ismatch:
            print(framename+': True')
        else:
            print(framename+": False")
    except:
        print(framename)

/tmp/ipykernel_33359/651115605.py:8: RuntimeWarning: coroutine 'get_feature_value' was never awaited
  detect_feature = get_feature_value(image=frame)
/tmp/ipykernel_33359/651115605.py:9: RuntimeWarning: coroutine 'get_feature_value' was never awaited
  base_feature = get_feature_value(image=base_img)


frame_985.jpg: False
frame_935.jpg: False
frame_375.jpg: False
frame_536.jpg: True
frame_648.jpg: False
frame_667.jpg: True
frame_346.jpg: False
frame_630.jpg: False
frame_978.jpg: False
frame_549.jpg: False
frame_632.jpg: False
frame_288.jpg: True
frame_817.jpg: True
frame_17.jpg: False
frame_937.jpg: False
frame_1054.jpg: False
frame_213.jpg
frame_778.jpg: False
frame_469.jpg: True
frame_779.jpg: False
frame_315.jpg: True
frame_506.jpg: True
frame_246.jpg: False
frame_920.jpg: False
frame_520.jpg: True
frame_560.jpg: False
frame_338.jpg: False
frame_1040.jpg: False
frame_522.jpg: True
frame_1074.jpg: False
frame_228.jpg: False
frame_264.jpg: False
frame_646.jpg: False
frame_993.jpg: True
frame_1090.jpg: False
frame_356.jpg: False
frame_582.jpg: True
frame_705.jpg: True
frame_849.jpg: False
frame_586.jpg: True
frame_385.jpg: False
frame_745.jpg: False
frame_934.jpg: False
frame_148.jpg: True
frame_507.jpg: True
frame_325.jpg: False
frame_397.jpg: False
frame_709.jpg: True
frame_610.jp

In [ ]:
# base_img = cv2.imread('/home/guohao826/AppAlgorithmFace/process_file/imgs/2.png')
# img = cv2.imread('/home/guohao826/AppAlgorithmFace/process_file/imgs/1.png')
# detect_feature = await get_feature_value(image=img)
# base_feature = await get_feature_value(image=base_img)
# data_score = get_compare_score(detect_feature, base_feature)
#     # print(data_score)
# data_ismatch = True if data_score >= Config.FEATURE_MATCH_THRESHOLD else False
# # data_ismatch = process_frame(img, base_img)
# if data_ismatch:
#     print('jpg : True')
# else:
#     print("jpg : False")